In [1]:
import pandas as pd
import requests
from bs4 import BeautifulSoup

In [2]:
kozig_uri = u'https://kozigallas.gov.hu/pages/jobviewer.aspx?ID={}'
chrome_header = ('Mozilla/5.0 (Windows NT 6.2; Win64; x64) '
                 'AppleWebKit/537.36 (KHTML, like Gecko) '
                 'Chrome/32.0.1667.0 Safari/537.36')

In [3]:
def init_session():
    session = requests.Session()
    response = session.get('https://kozigallas.gov.hu/publicsearch.aspx',
                           headers={'User-agent': chrome_header})
    return session

In [4]:
def get_page(session, url):
    return session.get(url, headers={'User-agent': chrome_header})

In [5]:
def get_joblists(session):
    response = get_page(session, 'https://kozigallas.gov.hu/publicsearch.aspx')
    soup = BeautifulSoup(response.content, 'html.parser')
    
    max_page = soup.find('span', {'id': ('ctl00_ContentPlaceHolder1_'
                                         'JobSearchForm1_JobList1_lblPageCount2')})
    max_page = int(max_page.getText().split(':')[-1].strip())
    print('Found {} page.'.format(max_page))
    
    search_uri = u'https://kozigallas.gov.hu/publicsearch.aspx?p={}'
    return [search_uri.format(page) for page in range(1, max_page+1)]

In [6]:
def extract_jobs(response):
    soup = BeautifulSoup(response.content, 'html.parser')
    table = soup.find('div', {'class': 'joblist'}).find('table')
    df = pd.read_html(table.prettify(), header=0)[0]
    df = df[df.columns[:-2]]
    df['job_id'] = [jid.get('href').split("'")[1] 
                    for jid in soup.findAll('a', {'class': 'jobapplication'})]
    return df

In [7]:
def read_job(session, job_id):
    print '.',
    response = session.get(kozig_uri.format(job_id))
    soup = BeautifulSoup(response.content, 'html.parser')
    tag = soup.find('p', {'class': 'MsoNormal'})
    return ' '.join([tag.getText().strip() for tag in soup.findAll('p', {'class': 'MsoNormal'})])

In [8]:
def scrape():
    with init_session() as session:
        print('joblist download')
        dfs = [extract_jobs(get_page(session, joblist))
               for joblist in get_joblists(session)]
        print('df concat')
        dfs = pd.concat(dfs)
        print('downloading {} jobs'.format(len(dfs)))
        dfs['text'] = dfs.apply(lambda row: read_job(session, row['job_id']),
                                axis=1)
        print('done.')
    return dfs

In [ ]:
df = scrape()

In [10]:
df.to_csv('data/kozigallas20170727.csv', encoding='utf-8')
df.to_excel('data/kozigallas20170727.xlsx', encoding='utf-8')

---

In [2]:
df2 = pd.read_csv('data/kozigallas20170727.csv', index_col=0)
df2.head()

,Jelentkezési határidő,Állás megnevezése,Közzétevő,Munkavégzés helye,Típus,job_id,text
0,2017. július 27.,diplomás ápoló,Csongrád Városi Önkormányzat,"Csongrád megye , Csongrád",Közalkalmazott,rk5d2q17c3,Csongrád Városi Önkormányzat Csongrád Városi ...
1,2017. július 27.,Magyar nyelv és irodalom - bármely szakos taná...,Tatabányai Tankerületi Központ Bárdos László G...,"Komárom-Esztergom megye , Tatabánya",Közalkalmazott,rlye3za7h5,Tatabányai Tankerületi Központ Bárdos László ...
2,2017. július 27.,magyar - történelem szakos középiskolai tanár,Tatabányai Tankerületi Központ Bárdos László G...,"Komárom-Esztergom megye , Tatabánya",Közalkalmazott,rlye2qaxf1,Tatabányai Tankerületi Központ Bárdos László ...
3,2017. július 27.,tanár - vendéglátás 310773.,Karcagi Szakképzési Centrum,"Jász-Nagykun-Szolnok megye , Tiszafüred",Közalkalmazott,rlcj9rgxd6,Karcagi Szakképzési Centrum Karcagi Szakképzé...
4,2017. július 27.,raktáros,Országos Korányi Pulmonológiai Intézet - Budapest,NaN,Közalkalmazott,rlvd8eg3a4,Országos Korányi Pulmonológiai Intézet - Buda...


In [8]:
df2.text.str.len().describe()

count     6477.000000
mean      6117.330863
std       1854.319251
min       3296.000000
25%       4886.000000
50%       5728.000000
75%       6800.000000
max      24395.000000
Name: text, dtype: float64

In [3]:
df3 = pd.read_excel('data/kozigallas20170727.xlsx', index_col=0)
df3.head()

In [7]:
df3.text.str.len().describe()

count     6477.000000
mean      5523.809480
std       1668.504575
min       2978.000000
25%       4413.000000
50%       5174.000000
75%       6130.000000
max      21836.000000
Name: text, dtype: float64

In [9]:
df2.to_excel('data/kozigallas20170727_fromcsv.xlsx', encoding='utf-8')

---

In [ ]:
data = pd.read_excel('/Users/fulibacsi/workspace/DATA/kozigallas20170725.xlsx').set_index('job_id')

In [ ]:
unwanted = ['\xa0', '\r', '\n']
def clear(text):
    cleared = ''.join([char for char in text if char not in unwanted])
    return ' '.join(cleared.split())

def process(itemlist):
    items = itemlist.strip().split('•')
    return [item.strip() for item in items if item]

def split(text):
    text = clear(text)
    header, tasks = text.split('A munkakörbe tartozó, illetve a vezetői megbízással járó lényeges feladatok:')
    tasks, footer = tasks.split('Illetmény és juttatások:')
    body, req = footer.split('Pályázati feltételek:')
    req, plus = req.split('A pályázat elbírálásánál előnyt jelent:')
    plus, footer = plus.split('A pályázat részeként benyújtandó iratok, igazolások:')
    etc = ' '.join([header.strip(), body.strip(), footer.strip()])
    return process(tasks), process(req), process(plus), etc

tasks, req, plus, etc = split(data.loc['rjyg6qkbb3', 'text'])

def extract(text):
    tasks, req, plus, etc = split(text)
    return {'tasks': tasks, 'req': req, 'plus': plus}

In [ ]:
data.text.apply(extract)